In [2]:
import pandas as pd
import numpy as np
import pickle
from keras.models import Model, Sequential
from keras.applications import resnet50
from keras.applications import vgg16
from keras.preprocessing import image
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
from keras.layers import *
from keras import layers
import keras
from keras.callbacks import EarlyStopping, Callback
# from skimage.io import imread
import sklearn
K.set_image_data_format('channels_last')

path = '../Data/'

class_emb = pd.read_csv(path + '/DatasetA_train_20180813/class_wordembeddings.txt', 
                        index_col = 0, sep = ' ', header = None)
class_emb.index.name = 'class_name'
# class_emb_vec = pd.DataFrame(index = class_emb.index)
class_emb = class_emb.apply(lambda s: np.array([float(x) for x in s]), axis = 1)

class_id_to_name = pd.read_csv(path + '/DatasetA_train_20180813/label_list.txt', 
                               index_col = 'class_name', sep = '\t', header = None, names = ['class_id', 'class_name'])

attr_list = pd.read_csv(path + '/DatasetA_train_20180813/attribute_list.txt', index_col = 0, sep = '\t', header = None)

attributes_per_class = pd.read_csv(path + '/DatasetA_train_20180813/attributes_per_class.txt', 
                                   index_col = 0, sep = '\t', header = None)
attributes_per_class.index.name = 'class_id'
attributes_per_class = attributes_per_class.apply(lambda s: np.array([float(x) for x in s]), axis = 1)

class_id_emb_attr = class_id_to_name.copy()
class_id_emb_attr['emb'] = class_emb
class_id_emb_attr.reset_index(inplace = True)
class_id_emb_attr.set_index('class_id', inplace = True)
class_id_emb_attr['attr'] = attributes_per_class

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# train_data = pd.read_csv(path + '/DatasetA_train_20180813/train.txt', index_col = 'class_id', 
#                          sep = '\t', header = None, names = ['image_id', 'class_id'])
# train_data = train_data.merge(class_id_emb_attr, how = 'left', on = 'class_id')

# imag_path = path + r'/DatasetA_train_20180813/train/'
# def read_image(image_id):
#     img = image.load_img(imag_path + image_id)
#     img = image.img_to_array(img)
# #     img= vgg16.preprocess_input(image.img_to_array(img))
#     return img
# train_data['img'] = train_data['image_id'].apply(lambda id: read_image(id))

# train_data.reset_index(inplace = True)

with open(path + 'train_img.pickle', 'rb') as handle:
    train_data = pickle.load(handle)

In [7]:
train_data.head()

,class_id,image_id,class_name,emb,attr,img
0,ZJL1,a6394b0f513290f4651cc46792e5ac86.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[20.0, 33.0, 7.0], [19.0, 32.0, 6.0], [22.0,..."
1,ZJL1,2fb89ef2ace869d3eb3bdd3afe184e1c.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[140.0, 45.0, 51.0], [144.0, 47.0, 54.0], [1..."
2,ZJL1,eda9f3bef2bd8da038f6acbc8355fc25.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[81.0, 69.0, 21.0], [86.0, 74.0, 26.0], [85...."
3,ZJL1,7d93ef45972154aae150b4f9980a79c0.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[16.0, 14.0, 15.0], [18.0, 16.0, 17.0], [19...."
4,ZJL1,fb901b4f9a8e396c1d0155bccc5e5671.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[120.0, 124.0, 127.0], [89.0, 93.0, 96.0], [..."


In [49]:
imread(path + '/DatasetA_train_20180813/train/a6394b0f513290f4651cc46792e5ac86.jpeg').shape

(64, 64, 3)

In [11]:
category = train_data['class_id'].unique()
category_dict = dict((category[i], i) for i in range(category.shape[0]))

vgg_model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = train_data['class_id'].unique().shape[0])


# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_y = pickle.load(handle)

# input = Input(shape = (2048, ))
# dense = Dense(2048, activation="relu")(input)
# dense = Dropout(0.5)(dense)
# dense = Dense(2048, activation="relu")(input)
# dense = Dense(1024, activation="relu")(input)
# dense = Dense(category.shape[0], activation="softmax")(input)

# vgg_model = Model(input, dense)

# vgg_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

# print (vgg_model.summary())

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)
# train_data['target'] = list(train_image_feature_map)

def small_vgg(img_input_shape, classes):
    # Block 1
    img_input = Input(shape = (img_input_shape))
    x = layers.Conv2D(32, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(32, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    # Block 3
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
#     # Block 4
#     x = layers.Conv2D(128, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv1')(x)
#     x = layers.Conv2D(128, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv2')(x)
#     x = layers.Conv2D(128, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block4_conv3')(x)
#     x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
#     # Block 5
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv1')(x)
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv2')(x)
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv3')(x)
#     x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    # Classification block
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(1024, activation='relu', name='fc1')(x)
#     x = layers.Dense(512, activation='relu', name='fc2')(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)
    
    model = Model(img_input, x)
    print (model.summary())
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    
    return model
    
    
train_img = extract_array_from_series(train_data['img'])
train_img = vgg16.preprocess_input(train_img)
OneHotEncoder = sklearn.preprocessing.OneHotEncoder()
train_target = train_data['class_id'].apply(lambda id: category_dict[id]).values
train_target = OneHotEncoder.fit_transform(np.reshape(train_target, (-1, 1))).toarray()

fold = 5
kf = KFold(n_splits=fold, shuffle=True)
num_fold = 0

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)

for train_index, test_index in kf.split(train_img):
    train_part_img = train_img[train_index]
    validate_part_img = train_img[test_index]
    
    train_part_target = train_target[train_index]
    validate_part_target = train_target[test_index]
    
    print ('Train target nunique: ', np.unique(np.argwhere(train_part_target == 1)[:, 1]).shape[0], 
           'Validate target nuique: ', np.unique(np.argwhere(validate_part_target == 1)[:, 1]).shape[0])
#     model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = category.shape[0])
#     model = resnet50.ResNet50(include_top=True, weights = None, input_shape=(224, 224, 3), classes = category.shape[0])
#     print (model.summary())
#     model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    img_model = small_vgg(img_input_shape = (64, 64, 3), classes = train_target.shape[1])
    img_model.fit(train_img, train_target, validation_data=(validate_part_img, validate_part_target), 
                  epochs=5, batch_size = 64, shuffle=True, verbose = 1)
    break

Train target nunique:  190 Validate target nuique:  190
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 32)        896       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 32)        9248      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 64)        18496     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block2_pool (MaxPool

In [13]:
img_model.save(path + '/small_vgg.h5')

In [186]:
# np.asarray(list(train_data['img'])).shape
# train_data[train_data['class_id'] == 'ZJL1']
# train_data['class_id'].value_counts().hist()
# train_data.head()
# class_id_emb_attr.iloc[0].name
# resnet50_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))
img = image.load_img(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = vgg16.preprocess_input(img)
preds = vgg_model.predict(np.expand_dims(img, axis=0))
print('Predicted:', vgg16.decode_predictions(preds, top=3)[0])
# imread(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg')
# image.img_to_array(img).shape

Predicted: [('n03598930', 'jigsaw_puzzle', 0.023861792), ('n02095314', 'wire-haired_fox_terrier', 0.022931756), ('n04589890', 'window_screen', 0.021361042)]


In [30]:
def create_dnn():
    attr_input = Input(shape = (30,))
    word_emb = Input(shape = (300,))
    imag_classifier = Input(shape = (1024,))

    attr_dense = Dense(2048, activation="relu")(attr_input)
    attr_dense = Dense(1024, activation="relu")(attr_input)
#     attr_dense = Dense(300, activation="relu")(attr_dense)
    # attr_dense = Dense(512, activation="relu")(attr_dense)

    attr_word_emb = Concatenate()([word_emb, attr_dense])
    
#     attr_word_emb_dense = attr_word_emb
#     attr_word_emb_dense = Dense(4096, activation="relu")(attr_word_emb_dense)
    attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)
    attr_word_emb_dense = Dense(1024, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(512, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
    # attr_word_emb_dense = Dense(2048, activation="sigmoid", kernel_regularizer=keras.regularizers.l2(0.01))(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(1024, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(4096, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(8192, activation="relu")(attr_word_emb_dense)

    # mse = Dense(6, activation="softmax")(dense_output)
#     vgg_input = vgg_model.input
#     vgg_output = Flatten()(vgg_model.output)
#     vgg_output = Dense(1024, activation="sigmoid")(vgg_output)

    # mse_loss = keras.losses.mean_squared_error(vgg_output, attr_word_emb_dense)

    model = Model([attr_input, word_emb], outputs = attr_word_emb_dense) #, vgg_output])
    # model.add_loss(mse_loss)
    model.compile(optimizer='adam', loss='mean_squared_error')
#     model.summary()
    return model

def find_nearest_class(class_id_emb_attr, model, eval_df, img_feature_map = None):
    if img_feature_map is None:
        img_feature_map = np.reshape(vgg_model.predict(np.asarray(list(eval_df['img'])), verbose = 1), (eval_img.shape[0], -1))
    cand_feature_map = model.predict(create_dnn_data(class_id_emb_attr))
    # img_feature_map = np.repeat(img_feature_map, [cand_feature_map.shape[0]] * img_feature_map.shape[0], axis = 0)
    nearest_class_id = ['ZJL'] * eval_df.shape[0]
    for i in range(img_feature_map.shape[0]):
        img = img_feature_map[i]
        dis = np.linalg.norm(img - cand_feature_map, axis = 1)
        min_ind = np.where(dis == np.amin(dis))[0]
        if len(min_ind) > 1:
            print ('eval img id: ', eval_df[i]['class_id'], 'has multiple best candidates: ', len(min_ind))
#         print (i, img, class_id_emb_attr.iloc[min_ind[0]].name)
        nearest_class_id[i] = class_id_emb_attr.iloc[min_ind[0]].name
#     print (nearest_class_id)
    return np.asarray(nearest_class_id)
    
    
def extract_array_from_series(s):
    return np.asarray(list(s))
    
def create_dnn_data(df):
    return [extract_array_from_series(df['attr']), extract_array_from_series(df['emb'])]
    
class AccuracyEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_interval = 1000000, verbose = 2, \
            scores = [], class_id_emb_attr = None, eval_df = None):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        # print("y_val shape:{0}".format(self.y_val.shape))
        self.batch_interval = batch_interval
        self.verbose = verbose
        self.scores = scores
        self.class_id_emb_attr = class_id_emb_attr
        self.eval_df = eval_df
#         self.img_feature_map = self.y_val

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            pred_nearest_class_id = find_nearest_class(self.class_id_emb_attr, self.model, self.eval_df, self.y_val)
            true_class_id = self.eval_df['class_id'].values
#             print (pred_nearest_class_id, "\n", true_class_id)
            right_num = np.sum(pred_nearest_class_id == true_class_id)
            score = right_num / true_class_id.shape[0]
            self.scores.append("epoch:{0} {1}".format(epoch + 1, score))
            print("\n Accuracy epoch: %d - score: %.6f \n %d/%d" % (epoch+1, score, right_num, true_class_id.shape[0]))
    
    def on_batch_end(self, batch, logs={}):
        return
        if(self.verbose >= 2) and (batch % self.batch_interval == 0):
            # y_pred = self.model.predict(self.X_val, verbose=0)
            # loss = metrics.log_loss(self.y_val, y_pred)
            print("Hi! on_batch_end() , batch=",batch,",logs:",logs)
            # print("Valide size=",y_pred.shape[0], "  Valide loss=",loss)

In [15]:
img_model_flat = Model(input = img_model.input, output = img_model.get_layer(name = 'fc1').output)
train_data['target'] = list(img_model_flat.predict(train_img, verbose = 1))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """Entry point for launching an IPython kernel.


38221/38221 [==============================] - 13s    


In [31]:
#data.loc[class_id] #data_atten.loc[n]
fold = 5
kf = KFold(n_splits=fold, shuffle=True)
num_fold = 0
class_ids = train_data['class_id'].unique() #[:5]

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)

for train_index, test_index in kf.split(class_ids):
    train_part_id = class_ids[train_index]
    validate_part_id = class_ids[test_index]
    
    train_part_df = train_data[train_data['class_id'].isin(train_part_id)]
    validate_part_df = train_data[train_data['class_id'].isin(validate_part_id)]

    train_part_data = create_dnn_data(train_part_df)
    train_part_target = extract_array_from_series(train_part_df['target'])
    
    validate_part_data = create_dnn_data(validate_part_df)
    validate_part_target = extract_array_from_series(validate_part_df['target'])
    
    print('fold: %d th train :-)' % (num_fold))
    print('Train size: {} Valide size: {}'.format(train_part_df.shape[0], validate_part_df.shape[0]))

    callbacks = [
            EarlyStopping(monitor='val_loss', patience=30, verbose=0),
            AccuracyEvaluation(validation_data=(validate_part_data, validate_part_target), interval=1, \
                class_id_emb_attr = class_id_emb_attr, eval_df = validate_part_df)
            ]
    
    model = create_dnn()
    if num_fold == 0:
        print (model.summary())
    model.fit(train_part_data, train_part_target,  validation_data=(validate_part_data, validate_part_target),
                  epochs=100, batch_size = 64, shuffle=True, verbose = 1, callbacks=callbacks)
    num_fold += 1

fold: 0 th train :-)
Train size: 30582 Valide size: 7639
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_39 (InputLayer)            (None, 30)            0                                            
____________________________________________________________________________________________________
input_40 (InputLayer)            (None, 300)           0                                            
____________________________________________________________________________________________________
dense_27 (Dense)                 (None, 1024)          31744       input_39[0][0]                   
____________________________________________________________________________________________________
concatenate_5 (Concatenate)      (None, 1324)          0           input_40[0][0]                   
                                  

KeyboardInterrupt: 

In [19]:
np.ndenumerate(data.columns.values)

In [ ]:
 
data_test = open("D:\\data\\pic\\DatasetA_test_20180813\\DatasetA_test\\image.txt")
data_test = data_test.readlines()
path_test = r'D:\data\pic\DatasetA_test_20180813\DatasetA_test\test\\'
length = len(data_test)
data_test_x = np.zeros((length, 64, 64, 3))
for i in range(length):
    m = data_test[i][:-1]
    img = image.load_img(path_test + m)
    data_test_x[i] = image.img_to_array(img)
 
res = model.predict(data_test_x)
 
#欧氏距离：
data_atten_res = []
for i in range(160,241):
    key = 'ZJL' + str(i)
    data_atten_res.append(np.hstack((data_atten.loc[key][1:8], data_atten.loc[key][16:20], data_atten.loc[key][20:26])))
 
vec_result = []#np.zores((len(res[0]), 18))
for i in range(len(res[0])):
    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i])))
dis_list = []
 
for j in range(len(data_atten_res)):
    vec1 = vec_result[0]
    vec2 = data_atten_res[j]
    dist = np.linalg.norm(vec1 - vec2)
    dis_list.append(dist)
print("witch is : "+ str(dis_list.index(min(dis_list))) + " and min dis = " + str(min(dis_list)))